In [9]:
#export
import json
import sys
import os
import os.path
from pathlib import Path
from subprocess import call
import subprocess
import sys # system libraries, like arguments (argv)
import re # regular expressions
import pandas as pd
import glob
import os.path
import numpy as np
from pathlib import Path
from scipy import stats
import statsmodels as sm



from timeit import default_timer as timer
import time



In [10]:
#export
def removeFile(filename):
    try:
        os.remove(filename)
    except OSError:
        pass   

In [11]:
#export
def makeFolders(folder_list):
    for directory in folder_list:
        if not directory: # Make sure not an empty string ""
            continue
        else:
            if not os.path.exists(directory):
                os.makedirs(directory)

### Filter class (Filter contigs of interest)

In [12]:
#export
class JSON():
    def __init__(self, f, explicit, min_length = 100):
        self.f = f
        self.min_length = min_length
        self.json_empty = False
        self.num_contigs = 0
        with open(f) as fo:
            cd = json.load(fo) # cd is contig dictionary
            k = list(cd.keys())
            print(f'starting with {len(k)} contigs')
            if len(k) > 0:
                self.filename = cd[k[0]]["filename"]
                self.cd = cd
            else:
                self.json_empty = True
                self.cd = {}
                self.pe(f'*-**-**-**-**-**-**-**-**-**-* WARNING file Empty!!!')
            


## Concat Jsons after query and make gtf

In [22]:
#export
class Match():
    
    def __init__(self, 
             NF_out, 
             NF_path,
             col_data, 
             levels,
             explicit = True):

        self.explicit = explicit
        self.NF_out = NF_out
        self.NF_path = NF_path
        self.wkdir = f'{self.NF_out}/unmapped/final'
        self.Jlist_d = {}
        self.df_norm = pd.DataFrame()
        self.col_data = col_data
        self.df_col = pd.read_csv(self.col_data,sep="\t",names = ["sample","condition"])
        self.condition_list = self.df_col["condition"].unique()
#         self.levels = ["single","group","all",
#                        "dark_single_FG","dark_group_FA","dark_all_FG"]
        self.levels = levels
        self.toJSON()



    def toJSON(self):
        import ast
        

        for f in glob.glob(self.wkdir+"/jgi_df/*/*txt"):
#             print(f)
            
            l = len("_df.txt")
            name = os.path.basename(f)[:-l]
            fout = f[:-l]
            df = pd.read_csv(f,sep="\t")
#             print(df.head())
            df_J = df[["taxid","jgi_json"]]
            del df["jgi_json"]
            df.to_json(f"{fout}_temp.json",orient='index')
            
            with open(f"{fout}_temp.json") as fo:
                contig_d = json.load(fo)
                for c in contig_d.keys():
                    d = contig_d[c] 
                    taxid = d["taxid"]  
                    df_temp = df_J[df_J["taxid"]==taxid]
                    j = df_temp["jgi_json"].iloc[0]
                    jdic = ast.literal_eval(j)
                    d["jgi_json"] = jdic
                    d["filename"] = name

                with open(f"{fout}.json", "w") as write_file:
                    json.dump(contig_d, write_file)
            removeFile(f"{fout}_temp.json")
  
        
    
              

    def makeDF(self,cd, level):
        print("Making df")
        # Add NA stuff to it
        for contig_key in cd:
            contig = cd[contig_key]
            contigname = contig["contig"]
            jgi = contig["jgi_json"]
            try:
                superkingdom = jgi["superkingdom"]
            except:
                jgi["superkingdom"] = {}
                jgi["superkingdom"]["name"] = "NA"
            try:
                kingdom = jgi["kingdom"]
            except:
                jgi["kingdom"] = {}
                jgi["kingdom"]["name"] = "NA"
            try:
                phylum = jgi["phylum"]
            except:
                jgi["phylum"] = {}
                jgi["phylum"]["name"] = "NA" 
            try:
                order = jgi["order"]
            except:
                jgi["order"] = {}
                jgi["order"]["name"] = "NA"    
            try:
                family = jgi["family"]
            except:
                jgi["family"] = {}
                jgi["family"]["name"] = "NA"    
            try:
                genus = jgi["genus"]
            except:
                jgi["genus"] = {}
                jgi["genus"]["name"] = "NA" 
            try:
                species = jgi["species"]
            except:
                jgi["species"] = {}
                jgi["species"]["name"] = "NA"                 

                # Update the contig list
        cd[contig_key]["jgi_json"] = jgi
     # Add to dataframe and write out
        df = pd.DataFrame.from_dict(cd).T

        if level == "all":
            df = df[["contig","blast_pident","blast_sseqid", "blast_evalue" ,"taxid" ,"blast_stitle","filename","length","fasta"]]
        else:
            df = df[["contig","taxid" ,"filename","length","fasta"]]
        
        df_final = pd.DataFrame()
        for contig_key in cd:
            contig = cd[contig_key]
            contigname = contig["contig"]
            jgi = contig["jgi_json"]
            dfJ = pd.DataFrame.from_dict(jgi)

            dfJ["contig"] = contigname
            dfJ = dfJ[["contig","name","superkingdom","kingdom","phylum","order","family","genus","species"]]
            dfJ = dfJ.loc[["name"]]


            df_final = pd.concat([df_final,dfJ])
        df = pd.merge(df_final,df,on="contig")       
        return df


    
    def writeDF(self):
#         makeFolders([f'{self.NF_out}/{self.title}'])
        for level in self.levels:
            f_pileup = ""
            if level == "all" or "dark_all_FG":
                f_pileup = f"{self.wkdir}/pileupCoverageNormalized/all_pileupCoverageNormalized.txt"
            

            
            df_pileup = pd.read_csv(f_pileup,sep="\t")

            df_final = pd.DataFrame()

            for J in self.Jlist_d[level]:
                df = self.makeDF(J.cd, level)
#                 df = pd.DataFrame.from_dict(J.cd).T

                df_final = pd.concat([df_final,df])
                
            df_final = pd.merge(df_final,df_pileup,on="contig")
            
            out = f"{self.wkdir}/pileupCoverageNormalizedMatched"
            makeFolders([out])
            f_df = f'{out}/{level}_pileupCoverageNormalizedMatched.txt'

            df_final.to_csv(f_df,sep="\t",index=None)
            
            
          
            if level == "dark_all_FG":
# /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/Z_RNAseq-Biome-master/NF_OUT/unmapped/final/darkbiome/lastdb/FINAL_Nohits/all_group/all/all_darkbiome_df.txt                
                f_df = f'{out}/{level}_NoHits_pileupCoverageNormalizedMatched.tsv'
                df_final = pd.read_csv(f"{self.wkdir}/darkbiome/lastdb/FINAL_Nohits/all_group/all/all_darkbiome_df.txt",sep="\t")

                df_final = pd.merge(df_final,df_pileup,on="contig")  
                df_final.to_csv(f_df,sep="\t",index=None)

            
            

 

In [26]:
NF = "/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/nextflow/RNAseq-Biome-Nextflow"

NF = "/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/Z_RNAseq-Biome-master"


NF = "/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/trem2/RNAseq-Biome-master"




NF_out = f"{NF}/NF_OUT" 
NF_path = NF
out_path = NF
col_data = f"{NF}/sample_table.txt"
levels = ["all","dark_all_FG"]
# levels = ["dark_all_FG"]




M = Match(NF_out=NF_out, 
             NF_path=NF_path,
             col_data = col_data, 
             levels = levels,
             explicit = True
            )




M.searchContigs()
M.writeDF()


Working on all --- 
*_*_*_*_*_*_*_ WARNING *_*_*_*_*_*_*_
No files in all had any contigs
Working on dark_all_FG --- 
*_*_*_*_*_*_*_ WARNING *_*_*_*_*_*_*_
No files in dark_all_FG had any contigs


FileNotFoundError: [Errno 2] File b'/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/trem2/RNAseq-Biome-master/NF_OUT/unmapped/final/pileupCoverageNormalized/all_pileupCoverageNormalized.txt' does not exist: b'/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/trem2/RNAseq-Biome-master/NF_OUT/unmapped/final/pileupCoverageNormalized/all_pileupCoverageNormalized.txt'

In [23]:
#export
import argparse
def parse_arguments():
        parser = argparse.ArgumentParser(description='filter quantify and graph')
        parser.add_argument('--Nextflow_Out', action= 'store', metavar='--Nextflow_Out') 
        parser.add_argument('--Nextflow_path', action= 'store', metavar='--Nextflow_path')
        
        args = parser.parse_args()
        return args


In [24]:
#export
if __name__=="__main__":
    print("Concatenating pileup into dataframes...")
    args = parse_arguments()

    
  
    NF_out = args.Nextflow_Out
    NF_path = args.Nextflow_path
    col_data = f"{NF_path}/sample_table.txt"

    
    levels = ["all","dark_all_FG"]
    M = Match(NF_out=NF_out, 
                 NF_path=NF_path,
                 col_data = col_data, 
                 levels = levels,
                )


    M.searchContigs()
    M.writeDF()



    
    

Concatenating pileup into dataframes...


usage: ipykernel_launcher.py [-h] [--Nextflow_Out --Nextflow_Out]
                             [--Nextflow_path --Nextflow_path]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/m/Library/Jupyter/runtime/kernel-28df6773-ac50-4ebb-a5cc-87b6c810090f.json


SystemExit: 2

In [28]:
import fire
!python notebook2script.py 3.0-PileupDataFrame.ipynb

Converted 3.0-PileupDataFrame.ipynb to nb_3.0-PileupDataFrame.py
